# Analyzing New York City employees' payroll database 

## Data Source: [NYC open data](https://data.cityofnewyork.us/City-Government/Citywide-Payroll-Data-Fiscal-Year-/k397-673e/data)

In [1]:
import pandas as pd
df = pd.read_csv('Citywide_Payroll_Data__Fiscal_Year_.csv')
df.columns = df.columns.str.replace(" ", "_")
df.columns = df.columns.str.replace("-", "_")
df.columns = df.columns.str.lower()
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

/Users/areena.arora/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
boroughs = ['QUEENS', 'MANHATTAN', 'BROOKLYN', 'BRONX']

In [ ]:
df = df[df.work_location_borough.isin(boroughs)]

In [ ]:
df.shape

Note: This reduced the dataset by 103,863 rows.

In [ ]:
df['total_pay'] = df.regular_gross_paid + df.total_ot_paid + df.total_other_pay

In [ ]:
df.sort_values(by='total_pay', ascending = False).head(10)

In [ ]:
df.total_pay.max()

In [ ]:
df.total_pay.min()

### Ignoring the negatives. Assuming this is people who owe money to the city. The dataset alone cannot answer this so leaving this to reporting

### Interesting to see Jose Morales made $650K in 2020 and is no longer on city payroll

In [ ]:
df.query('last_name == "MORALES"').query('first_name == "JOSE"').query('agency_name == "POLICE DEPARTMENT"').query('agency_start_date == "12/20/1998"')

### Let's dive into overtime now :') !

In [ ]:
df.sort_values(by='total_ot_paid', ascending = False).head(10)

#### Five of these ten have the word 'plumber' in their title. Let's inspect further

In [ ]:
import re
df[df['title_description'].str.contains('.PLUMBER')== True].agency_name.value_counts()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].fiscal_year.value_counts()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].groupby(by='fiscal_year').total_ot_paid.mean()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].groupby(['fiscal_year', 'agency_name']).total_ot_paid.sum()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].groupby(['fiscal_year', 'agency_name']).total_pay.sum()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].total_ot_paid.mean()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].groupby(by='agency_name').total_ot_paid.sum()

In [ ]:
df[df['title_description'].str.contains('.PLUMBER')== True].groupby(by='agency_name').regular_gross_paid.sum()

## 📝 There were 52 plumbers on city payroll in 2021 and they made about \\$69,000 on average in overtime. Overall, the agency shelled out over $2 million in overtime just to its plumbers in 2021.

### NYCHA plumbers made \\$31,865 in base salary in total over five years. In overtime they made over $7.6 million

### Which agencies paid the most overtime and in which year?

In [ ]:
# pd.set_option('display.max_rows', None)
df.groupby(['agency_name', 'fiscal_year']).total_ot_paid.sum().reset_index().sort_values(by='total_ot_paid', ascending = False).head(10)

In [ ]:
df.groupby(by='agency_name').total_ot_paid.sum().reset_index().sort_values(by='total_ot_paid', ascending = False).head(5)

In [ ]:
df.query('agency_name == "NYC HOUSING AUTHORITY"').groupby(by='fiscal_year').total_ot_paid.sum()

In [ ]:
from plotnine import *

In [ ]:
# (
#     ggplot(df,
#         aes('total_ot_paid', 'ot_hours'))
#         + geom_point(aes(color='work_location_borough', size=3))
#         + theme(figure_size=(12, 6))
#         + theme_bw()
#         + labs(
#             title = "Range of overtime hours worked vs overtime earned by borough",
#             y = "Overtime hours worked",
#             x = "Overtime earned"
#         )
# )

# Let's zoom into those who made over $200K in overtime

In [ ]:
(
    ggplot(df.sort_values(by='total_ot_paid', ascending = False).head(10),
        aes('total_ot_paid', 'ot_hours'))
        + geom_point(aes(color='agency_name', size=6))
        + theme(figure_size=(8, 5))
        + theme_bw()
        + labs(
            title = "Six out of 10 employees who made over $200K in overtime worked with NYC Housing Authority",
            y = "Overtime hours worked",
            x = "Overtime earned"
        )
)

### I'm also interested in seeing how people's regular hours compare with their overtime hours.

In [ ]:
(
    ggplot(df,
        aes('regular_hours', 'ot_hours'))
        + geom_point(aes(color='work_location_borough', size=6))
        + geom_abline(intercept = 0, slope = 1, color='black')
        + theme(figure_size=(8, 5))
        + theme_bw()
        + labs(
            title = "A lot of employees worked more hours in overtime than regular hours",
            y = "Overtime hours",
            x = "Regular hours"
        )
)

In [ ]:
df.query('regular_hours > 3000')

In [ ]:
df.query('ot_hours > 2000')

In [ ]:
df['hrs_diff'] = df.ot_hours - df.regular_hours

In [ ]:
df.query('regular_hours > 0').sort_values(by='hrs_diff', ascending = False).query('hrs_diff > 1').query('leave_status_as_of_june_30 == "CEASED"')

In [ ]:
# Final sanity check
df.fiscal_year.value_counts()

# Check-In #2 with Dhrumil!

Goal: zoom out, identify a few possible questions that will hopefully lead to story angles.

I have provided some tools below that you can use to find these stories. Just look for this emoji "👈". That will show you places in the code you can make small changes to look at different subsets of the data.

This analysis starts with an assumption. The amount of regular hours of work someone does is proportional to the amount of overtime hours they do. That's our model `ot_hours ~ regular_hours` (I later changed it to a squared term `ot_hours ~ I(regular_hours**2)` because that fit slightly better, but either one works just fine). **additional note:** _I later thanged it from `ot_hours ~ I(regular_hours**2)` to `ot_hours ~ I(regular_hours**5)` because that fits even better 😲...message me on slack if you need me to explain this..._

In [ ]:
# df.agency_start_date.dt - pd.Timestamp.today
# pd.to_datetime(df.agency_start_date, errors='coerce')\
#     .apply(lambda x: x - pd.to_datetime("today"))
df['agency_start_date'] = pd.to_datetime(df.agency_start_date, errors='coerce')
df['today'] = pd.to_datetime('today')
df['tenure'] = (df.today - df.agency_start_date).astype('timedelta64[Y]')
# df.agency_name.unique() 
df

In [ ]:
import statsmodels.formula.api as smf

# YOU CAN ADD FILTERS HERE IF YOU WANT TO LOOK INTO A PARTICULAR AGENCY
# Let's start with no filters
to_model = df # .query("agency_name=='DEPARTMENT OF CORRECTION'")

# title_description
# MODEL y=F(X) - which factors do you want to control for? 
# What do we think should explain the variance in overtime pay
model = smf.ols('ot_hours ~ regular_hours + tenure', data=to_model.query("agency_name=='POLICE DEPARTMENT'")) 
# note that I added a squared term because it fits better
# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression

results = model.fit()
display(results.summary())

# FINDING OUTLIERS
# + E (what is still unaccounted for once you have controlled for those factors)
outliers = to_model.query("agency_name=='POLICE DEPARTMENT'").assign(
    predicted = results.predict(),
    residulas = results.resid,
    residuals_z = results.resid / results.resid.std()
    )\
    .sort_values(by='residuals_z', ascending=False)

In [ ]:
jobs = [
    'LIEUTENANT D/A SPECIAL ASSIGNMENT',
    'TRAFFIC ENFORCEMENT AGENT',
    'POLICE CADET',
    'POLICE OFFICER D/A DETECTIVE 2ND GR',
    'POLICE OFFICER D/A DETECTIVE 1ST GR',
    'POLICE OFFICER'
]
(
    ggplot(to_model.query("agency_name=='POLICE DEPARTMENT'"), 
           aes(x='tenure', y='ot_hours/regular_hours')) +
        geom_point(alpha=.1)
)
    

In [ ]:
list(df[df.title_description.str.contains('CORR')].unique())

In [ ]:
(
    ggplot(df, aes(x='tenure', y='regular_hours')) +
        geom_point()
)
    

In [ ]:
df

## Who are the biggest outliers?

Here is a dataframe containing our friend KEVIN MURTHA. Who is Kevin Murtha was a great opening question for your analysis! But if we want to apply the same logic that we used to find Kevin Murtha, here are 25 other Kevin Murthas (note...25 is arbitrary, open the `overtime_outliers.csv` to see them all).


In [ ]:
outliers.to_csv('overtime_outliers.csv', index=False) # preivew the df below, open the CSV for details
outliers.head(25)[['fiscal_year', 'agency_name', 'first_name', 'last_name', 
                   'title_description',  'residuals_z', 'ot_hours', 'regular_gross_paid', 'total_ot_paid']]

In [ ]:
# select a person using the index in the DF above 
INDEX = 1705627 
person = outliers.loc[INDEX] # a person in a given year

# try and grab that person's records in other years (you may need to modify these queries)
person_in_all_years = outliers\
    .query('last_name==@person.last_name and first_name==@person.first_name')\
    .query('agency_name==@person.agency_name')\
    .query('agency_start_date==@person.agency_start_date')

# this will fail if we've identified more than one person 
# (5 records max 2017-2021) or if we identified 0 people
try:
    assert(len(person_in_all_years) <= 5)
    assert(len(person_in_all_years) > 0)
except: 
    print(f"person_in_all_years returned {len(person_in_all_years)} records (should be between 1 and 5)")
    print(f"please modify that query so it refers to only one person")
    raise

# get other people in that agency/job/year for comparison
similar_people = []
for i, my in person_in_all_years.iterrows():
    similar_people_this_year = df\
        .query("fiscal_year == @my.fiscal_year")\
        .query("agency_name == @my.agency_name")\
        .query("title_description == @my.title_description")
    similar_people.append(similar_people_this_year)
    
other_people_in_the_same_position = pd.concat(similar_people)

# PLOT!
plt = (
    ggplot(other_people_in_the_same_position, 
           aes(x='regular_gross_paid', y='total_ot_paid')) +
        geom_point(aes(color='leave_status_as_of_june_30'), alpha=.5) + 
        geom_point(aes(color='leave_status_as_of_june_30'), data=person_in_all_years, shape='x',size=5, stroke=2) + 
        geom_point(data=person_in_all_years, shape='x',size=5) + 
        geom_abline(intercept=0, slope=1, color='red') +
        facet_wrap(['fiscal_year', 'title_description'], nrow=1) +
        ggtitle(f"Who Is {person.first_name} {person.last_name} at {person.agency_name}?\n\n(Modify the INDEX at the top to change the chart and table below.)") + 
        theme(figure_size=(16,4))
)

# PLOT!
plt2 = (
    ggplot(other_people_in_the_same_position, 
           aes(x='regular_gross_paid', y='total_other_pay')) +
        geom_point(aes(color='leave_status_as_of_june_30'), alpha=.5) + 
        geom_point(aes(color='leave_status_as_of_june_30'), data=person_in_all_years, shape='x',size=5, stroke=2) + 
        geom_point(data=person_in_all_years, shape='x',size=5) + 
        facet_wrap(['fiscal_year', 'title_description'], nrow=1) +
        ggtitle(f"Who Is {person.first_name} {person.last_name} at {person.agency_name}?\n\n(Modify the INDEX at the top to change the chart and table below.)") + 
        theme(figure_size=(16,4))
)

display(plt) # show plot
display(plt2) # show plot2
print('Here is a preview of the dataframe below with the person you selected. Check out overtime_outliers.csv if you\'re curious about other data points or change the INDEX to someone else and rerun the cell')
display(person_in_all_years) # show table

Awesome! Now you can modify the `INDEX` variable in the dataframe and re-run the above cells to see Kevin Murtha-like people in other agencies. This could be a handful of leads for possible stories. Time to start reporting!

"Who is Kevin Murtha" was a great question...but who is.. OMAR MALCOLM? ROBERT PROCIDA? HOWARD KNOX? 

Modify the `INDEX` variable in the dataframe and re-run the above to look into any particular person relative to others in similar roles. It's probably also a good time to leave the data notebook and start reporting!!!

## What departments do they work in? 

⚠️ **methodological choice alert**

note: when you make this choice, you are drawing attention away from individual people who are outliers and drawing attention towards particular professions/agencies that are prone to work overtimne. So by filtering and getting `value_counts` in this way, something is lost. 

Take, for example MICHAEL GAGER, a MARINE ENGINEER for the FIRE DEPARTMENT `(residuals_z == 20.41)`. He is not represented in the professions with the most people with high residuals below. Or the police officer you identified earlier KEVIN MURTHA `(residuals_Z==20.52)`. He is also not represented below. 

See the note in the cell below about the thresholds.

In [ ]:
# These are the biggest outliers (z score greater than 10)
# however if you mess with that threshold, you'll get different agencies/titles
# so there may be some agencies with more outliers that are less...outlier-y
# that you don't capture until you change the threshold from say 10 to 3

THRESHOLD = 10 # 👉 try changing this to 3 and see what happens...
               # neither is "wrong", they just surface different things
               # higher threshold means you get the outlier-iest outliers
               # lower threshold means you get all the outliers
               # I wouldn't set it to less than 3 (at that point you're not really looking at outliers at all)
               #      z-score of less than 3 that would put you in the "fat" part of the bell curve...
outliers['residuals_above_threshold'] = outliers.residuals_z.apply(lambda x: f'residual_z>{THRESHOLD}' if x > THRESHOLD else f'residual_z<={THRESHOLD}')


agencies_containing_outliers = outliers\
    .pivot_table(index=['agency_name', 'title_description'], aggfunc='count', 
                 columns='residuals_above_threshold', values='first_name')\
    .fillna(0).astype(int).reset_index()\
    .assign(
        pct = lambda df: df[f'residual_z>{THRESHOLD}'] / (df[f'residual_z>{THRESHOLD}'] + df[f'residual_z<={THRESHOLD}']) * 100
    ).rename_axis(None, axis=1)

# display
COLUMN_TO_SORT_BY = f'residual_z>{THRESHOLD}'
agencies_containing_outliers\
    .sort_values(by=COLUMN_TO_SORT_BY, ascending=False)\
    .head(10)

Here is our story about PLUMBERs at NYC HOUSING AUTHORITY that you were able to identify. Looks like we're on the right track, since as you learned from your background reading, it seems like there was [some wrongdoing there](https://twitter.com/thecityny/status/1128017319437971456) around overtime pay. So these are some leads for other "Plumbers at NYCHA" style stories. Are there other professions/departments that show the same kind of patterns? Why Focus on PLUMBERs at NYC HOUSING AUTHORITY...instead of say MAINTENANCE WORKERs in the same department?

These residuals give us a clue about where else to look. For example, what's up at the DEPARTMENT OF CORRECTION? Lots of people there seem to be getting relatively more ot_hours compared to regular_hours. I wonder why?

In [ ]:
agencies_containing_outliers = agencies_containing_outliers\
    .assign(
        pct = lambda df: df[f'residual_z>{THRESHOLD}'] / (df[f'residual_z>{THRESHOLD}'] + df[f'residual_z<={THRESHOLD}']) * 100
    ).rename_axis(None, axis=1)

**note:** Raw number of outlier people are interesting, and I think it is worth investigating the DEPARTMENT OF CORRECTION. I would follow that lede and try to remake the charts for DEPARTMENT OF CORRECTION employees of interest. But the DEPARTMENT OF CORRECTION is also huge!

Above, I create a variable that helps us normalize for the size of the department so that we don't overlook stories of groups of people that are getting lots of overtime pay in smaller departments. Let's see what we find.

In [ ]:
agencies_containing_outliers\
    .sort_values(by='pct', ascending=False).head(10)

Hmm...24 out of 60 INSTITUTIONAL AIDEs at DEPT OF HEALTH/MENTAL HYGIENE are outliers to our model...I wonder why? I might want to check the spreadsheet for INSTITUTIONAL AIDEs. What does an INSTITUTIONAL AIDE do? How much do they earn? Maybe this is normal...maybe it's a story? What about some of these other folks that are logging overtime at other departments? 

You can use this information to go and find individual people of interest in the CSV file. Plug them into the chart at the top of this section and see what the norm is in their profession and department.

## Looking at data by department

Dhrumil is tired...here are a bunch of charts you can work with if you'd like. OR just stick to the chart above. It is pretty informative to look at a particular person in the context of their profession + department. Just note that everything I did above makes that methodological choice. Why are we not looking, for example, at electricians across all departments. I don't know...I just didn't get around to it.

What I have done, is use the regression to generalize the logic you used to get to the Kevin Murtha story and the plumbers in NYCHA story. 

In [ ]:
# Here is that list of outliers one more time

# it is sorted by the z-score of the residuals from the model
# so people who got more overtime relative to regular pay will be surfaced
to_view =  outliers.\
    sort_values(by='residuals_z', ascending=False)\
    [['agency_name','title_description','residuals_z', 'last_name','first_name', 'mid_init', 
         'agency_start_date', 'total_ot_paid', 'regular_gross_paid', 'total_other_pay']]

to_view.to_csv('to_view.csv') # look at this CSV file to check particular departments

# OR try some queries...
# to_view = to_view.query('agency_name=="DEPT OF CITYWIDE ADMIN SVCS"')
# to_view = to_view.query('title_description=="ELECTRICIAN"')
# to_view = to_view.query('total_ot_paid > 100000')

to_view.sort_values(by='total_ot_paid', ascending=False)

In [ ]:
df

# bunch of other charts...you can use these, or ignore them. 

In [ ]:
# %%R -w 1000 -h 500

# require('readr')
# require('tidyverse')
# require('ggrepel')

# # WANT TO LOOK AT A PARTICULAR SUBSET OF PEOPLE? 
# # FILTER THEM HERE!
# df <- read_csv('overtime_outliers.csv') %>%
#     filter(agency_name == 'NYC HOUSING AUTHORITY')

# # WANT TO HIGHLIGHT PARTICULAR DATA POINTS?  
# to_highlight <- df %>%
#     filter(grepl('PLUMBER', title_description)) # regex search

# # WANT TO LABEL PARTICULAR PEOPLE? 
# to_label <- df %>%
#     filter(total_ot_paid > regular_gross_paid)

# # plot
# ggplot(df, aes(x=regular_gross_paid, y=total_ot_paid)) +
#     aes(color=grepl('PLUMBER', title_description)) +
#     geom_point(alpha=.5) +
#     geom_abline(intercept=0, slope=1, color='red') + 
#     geom_label_repel(data = to_label, 
#                      aes(label=title_description), 
#                      min.segment.length = 0) + # geom_label_repel labels at least one of every type of label in data
#     facet_wrap(~fiscal_year)
    

In [ ]:
# # Same as above, but in python/plotnine (very slow)

# # WANT TO LOOK AT A PARTICULAR SUBSET OF PEOPLE? 
# to_plot = outliers.query("agency_name == 'NYC HOUSING AUTHORITY'")

# # WANT TO HIGHLIGHT PARTICULAR DATA POINTS?  
# highlight_query = "title_description.str.contains('PLUMBER')"
# to_highlight = to_plot.query(highlight_query)

# # WANT TO LABEL PARTICULAR PEOPLE? 
# to_label = to_highlight.query("residuals_z > 10")

# # https://stackoverflow.com/questions/57701052/how-do-i-use-adjust-text-with-plotnine
# adjust_text_settings = {'expand_points': (2, 2), 'arrowprops': {'arrowstyle': '->', 'color': 'red'}}

# display(
#     ggplot(df, 
#            aes(x='regular_gross_paid', y='total_ot_paid')) +
#         geom_point(alpha=.2) +
#         geom_point(data=to_highlight, color='red', alpha=1) +
#         geom_abline(intercept=0, slope=1, color='red') + 
#         geom_text(to_label, aes(label='first_name'), adjust_text=adjust_text_settings) +
#         facet_wrap('fiscal_year') + 
#         theme(figure_size = (16,9))
# )

# display(to_label)

## And what is going on at that agency overall?

Let's look at the bigger picture of that agency. Are they leaning more heavily on overtime workers? 

This is the kind of analysis that lets you do more big trend stories. I have a feeling that the previous two sections will guide you to more compelling stories, but this could be additional context if it is helpful.

In [ ]:


# # reads the same variables from above (to_plot)
# # but plots a histogram instead of a dot plot

# to_plot.query(f"residuals_above_threshold == 'residual_z>{THRESHOLD}'")\
#     .title_description\
#     .value_counts()
# # ggplot(to_plot, aes(x=ot_hours)) +
# #     geom_histogram() +
# #     facet_grid(title_description=='CORRECTION OFFICER'~fiscal_year)
    

In [ ]:
median_ot = df.pivot_table(index='agency_name', values='ot_hours', aggfunc=['mean'], columns='fiscal_year').reset_index()
median_ot['diff'] = median_ot[(     'mean', 2021)] - median_ot[(     'mean', 2017)]
median_ot[median_ot[(       'diff',   '')] > 0]
# median_ot#.query("agency_name=='DEPT OF HEALTH/MENTAL HYGIENE'")
# len(median_ot.sort_values(by='diff').query('diff>0'))
# (
# ggplot(median_ot.melt(id_vars='agency_name'))
#     + aes(x='fiscal_year', y='value', group='agency_name') 
#     + geom_line(aes(color='agency_name')) 
    
# )   

In [ ]:
pd.read_csv('highlighted.csv').query('title_description.str.contains("WARDEN")')